<a href="https://colab.research.google.com/github/YanhuaZhang516/memory-representation-pomdp/blob/main/lstm_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### import Tensosflow in to the program:

In [1]:
%tensorflow_version 1.x
import tensorflow as tf


TensorFlow 1.x selected.


In [2]:
import pandas as pd
import numpy as np

### (1) lstm_1
#### set the lstm by myself

#### 1. set some hyper-parameters 



In [3]:
# define some hyper-parameters about lstm

# Number of neurons in the hidden state varaibles
num_nodes= 128
# Number of data points in a batch we process in validation 每一批次训练多少样本

# and the number for training data set is 4* batch_size
batch_size_in = 4
batch_size_va = 1
# Number of time steps we unrolling for durinngg optimization
# Trucated-BPTT 中使用的时间步数，步数越高，性能越好，但也会增加内存开销和计算时间
num_unrolling = 10
output_size=1

dropout = 0.2 # we use dropout here to avoid overfitting problem

#### 2. define the parameters about LSTM
##### 1. input gate: ix, im, ib
##### 2. forget gate: 
##### 3. output gate: 

In [4]:
# set the placeholders for the training data and validaiton data
tf. reset_default_graph()
# Input gate: ix, im, ib(it) - How much memory to write to cell state
# tf.truncated_normal(shape, mean, stddev) 
input_size = 2
output_size = 1
# connect the current input to the input gate
ix = tf.Variable(tf.random.truncated_normal([input_size, num_nodes], stddev = 0.02))
# connect the previous hidden state to the input gate

im = tf.Variable(tf.random.truncated_normal([num_nodes, num_nodes], stddev = 0.02))
# bias of the input gate
ib = tf.Variable(tf.random.uniform([1*batch_size_in, num_nodes], -0.02, 0.02))

# forget gate: fx, fm, fb(ft) - How much memory to discard from cell state
fx = tf.Variable(tf.random.truncated_normal([input_size, num_nodes], stddev = 0.02))
# connect the previous hidden state to the forget gate
fm = tf.Variable(tf.random.truncated_normal([num_nodes, num_nodes], stddev = 0.02))
# bias of the forget gate
fb = tf.Variable(tf.random.uniform([1*batch_size_in, num_nodes], -0.02, 0.02))

# Candidate value(c~t）- Used to compute the current cell state

cx = tf.Variable(tf.random.truncated_normal([input_size, num_nodes], stddev = 0.02))
# connect the previous hidden state to the candidate
cm = tf.Variable(tf.random.truncated_normal([num_nodes, num_nodes], stddev = 0.02))
# bias of the candidate
cb = tf.Variable(tf.random.uniform([1*batch_size_in, num_nodes], -0.02, 0.02))

# Output gate - How much memory to output from the cell state

ox = tf.Variable(tf.random.truncated_normal([input_size, num_nodes], stddev = 0.02))
# connect the previous hidden state to the candidate
om = tf.Variable(tf.random.truncated_normal([num_nodes, num_nodes], stddev = 0.02))
# bias of the candidate
ob = tf.Variable(tf.random.uniform([1*batch_size_in, num_nodes], -0.02, 0.02))

# In the end, we get output y through softmax
w = tf.Variable(tf.random.truncated_normal([num_nodes, output_size], stddev = 0.02))
b = tf.Variable(tf.random.uniform([output_size], -0.02, 0.02))

# Variables saving state across unrollings

# Hiddens state
saved_output = tf.Variable(tf.zeros([batch_size_in,num_nodes]), trainable= False, name='train_hidden')
# cell state
saved_state = tf.Variable(tf.zeros([batch_size_in,num_nodes]), trainable= False, name='train_cell')
# prediction state
saved_labels = tf.Variable(tf.zeros([batch_size_in,output_size]), trainable= False, name='train_labels')

# same varaibles for validation phase
saved_validate_output = tf.Variable(tf.zeros([batch_size_va,num_nodes]), trainable= False, name='validate_hidden')
saved_validate_state = tf.Variable(tf.zeros([batch_size_va,num_nodes]), trainable= False, name='validate_cell')

# same varaibles for test phase
saved_test_output = tf.Variable(tf.zeros([1,num_nodes]), trainable= False, name='test_hidden')
saved_test_state = tf.Variable(tf.zeros([1,num_nodes]), trainable= False, name='test_cell')

#### 3. define the LSTM cell

In [5]:
def lstm_cell(input_x , hidden_state , cell_state):
  """
  input:
  input_x: The input data at time t, the size is [batchsize, input_size]
  ix:[input_size, num_nodes]
  
  input_size:[obs, action]
  hidden_state: the hidden state h_(t-1) from the previous time t-1
  cell_state: the cell state c_(t-1) from the previous timt t-1

  output: return the output y_t , cell state c_t
  """

  input_gate = tf.sigmoid(tf.matmul(input_x, ix)+ tf.matmul(hidden_state,im) + ib)

  forget_gate =tf.sigmoid(tf.matmul(input_x, fx)+ tf.matmul(hidden_state,fm) + fb)
  # the candiate cell state c~t
  update = tf.matmul(input_x, cx) + tf.matmul(hidden_state, cm) + cb
  # the current cell state c_t
  c_t = forget_gate* cell_state + input_gate * tf.tanh(update)

  output_gate = tf.sigmoid(tf.matmul(input_x, ox) + tf.matmul(hidden_state, om) + ob)
  # new hidden state
  h_t = output_gate *tf.tanh(c_t)
  
  return h_t, c_t
  

In [6]:
# test the function of lstm_cell:
input_x=np.float32(np.random.rand(8).reshape(4,2))
hidden_state = np.ones((4,128),dtype=np.float32)
cell_state = np.ones((4,128),dtype=np.float32)
update = tf.matmul(input_x, cx) + tf.matmul(hidden_state, cm) + cb

h_t, c_t = lstm_cell(input_x , hidden_state , cell_state)

### Defining Inputs and Outputs
Here, we define two different types of inputs


*   Training inputs (batch_size>1, with unrolling)
*   Validation inputs (batch_size > 1, with unrolling)
*   Test inputs (batch_size =1, no unrolling)

Trainig set: Validation set =4:1





In [7]:
train_inputs, train_labels = [], []
valid_inputs, valid_labels = [], []

# defining unrolled training data placeholders 按时间展开
for x in range(num_unrolling):
  train_inputs.append(tf.placeholder(tf.float32, shape=[batch_size_in, input_size], name = 'train_inputs_%d'%x))
  train_labels.append(tf.placeholder(tf.float32, shape=[batch_size_in, 1], name = 'train_labels_%d'%x))
  # validation data placeholders
  valid_inputs = tf.placeholder(tf.float32, shape=[batch_size_va, input_size], name = 'validate_inputs')
  valid_labels = tf.placeholder(tf.float32, shape=[batch_size_va, 1], name = 'validate_labels')

test_input = tf.placeholder(tf.float32, shape=[1])

#### load the data

In [9]:
df = pd.read_csv('trajectory_10_grids.csv')
df = df.apply(pd.to_numeric, errors='ignore')

In [10]:
print(df.dtypes)

Unnamed: 0     int64
trajectory     int64
inputs        object
state          int64
dtype: object


In [11]:
def generate_batch_data(num_unrolling):
  batch_inputs=[]
  batch_labels=[]
  for i in range(num_unrolling):
    # time steps
    inputs=[]
    labels=[]
    for j in range(4):
      # batch size 
      action= np.float32(df.loc[df['trajectory']==(j+1)]['inputs'][250*j+i][1])
      obs = np.float32(df.loc[df['trajectory']==(j+1)]['inputs'][250*j+i][3])
      label = np.float32(df.loc[df['trajectory']==(j+1)]['state'][250*j+i])
      input=[action, obs]
      inputs.append(input)
      
      labels.append(label)
 
    inputs=np.array(inputs)
    labels = np.array(labels)
    batch_inputs.append(inputs)
    batch_labels.append(labels)


  return batch_inputs, batch_labels

In [12]:
# test the batch size 
batch_inputs, batch_labels=generate_batch_data(num_unrolling=10)

In [17]:
def train_lstm(lr, data_inputs, data_labels):
 
  outputs = []
  # the prediction part
  pre_outputs = []
  labels = []
  # at each step per unrolling
  # hidden state (batch_size, 128)
  output = saved_output
  # cell state (batch_size, 128)
  cell_state = saved_state
  label = saved_labels

  # compute the hidden state (output) and cell state (state)
  # recursively for all the steps in unrolling
  for i in range(len(train_inputs)):
    # the output for 1 time step
    output, cell_state = lstm_cell(input_x=train_inputs[i], hidden_state=output, cell_state= cell_state)
    output = tf.nn.dropout(output, keep_prob=1.0-dropout)
    # append each computed output value
    pre_output = tf.nn.softmax(tf.matmul(output, w) +b)
    pre_outputs.append(pre_output)

  # we add control dependencies here to gurantee the right oder of calculation
  with tf.control_dependencies([saved_output.assign(output), saved_state.assign(cell_state)]): 

    loss = tf.reduce_mean(tf.square(tf.reshape(pre_outputs,[-1]) - tf.reshape(train_labels,[-1])))
  
  # Adam Optimizer, and gradient clipping, here we use clipping to avoid the gradien exploation
  optimizer = tf.train.AdamOptimizer(learning_rate=lr)
  gradients, variables = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 5.0)
  # update the weights and bias
  train_op = optimizer.apply_gradients(zip(gradients, variables))
  # save the model 
  saver= tf.train.Saver(tf.global_variables())

  # generate the feed_dict
  feed_dict = {}
  for ui, (data,label) in enumerate(zip(data_inputs, data_labels)):
    feed_dict[train_inputs[ui]]=data
    feed_dict[train_labels[ui]] = label.reshape(4,1)

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # reset the training for 1000 times
    for i in range(1000):
      
      _, loss_ = sess.run([train_op, loss], feed_dict = feed_dict)
      # each 10 steps save once the parameters
      if i % 10==0:
        print(i, loss_)
        print("save the model:", saver.save(sess,'lstm.model'))
   
          

In [18]:
train_lstm(lr=1.5e-5,data_inputs=batch_inputs, data_labels=batch_labels)

0 209.125
save the model: lstm.model
10 209.125
save the model: lstm.model
20 209.125
save the model: lstm.model
30 209.125
save the model: lstm.model
40 209.125
save the model: lstm.model
50 209.125
save the model: lstm.model
60 209.125
save the model: lstm.model
70 209.125
save the model: lstm.model
80 209.125
save the model: lstm.model
90 209.125
save the model: lstm.model
100 209.125
save the model: lstm.model
110 209.125
save the model: lstm.model
120 209.125
save the model: lstm.model
130 209.125
save the model: lstm.model
140 209.125
save the model: lstm.model
150 209.125
save the model: lstm.model
160 209.125
save the model: lstm.model
170 209.125
save the model: lstm.model
180 209.125
save the model: lstm.model
190 209.125
save the model: lstm.model
200 209.125
save the model: lstm.model
210 209.125
save the model: lstm.model
220 209.125
save the model: lstm.model
230 209.125
save the model: lstm.model
240 209.125
save the model: lstm.model
250 209.125
save the model: lstm.mod

In [ ]:
# validation part
def prediction(inputs,labels):
  outputs=[]
  output = saved_validate_output
  cell_state = saved_validate_state
  for i in range(len(inputs)):
    output, cell_state = lstm_cell(input_x=i, hidden_state=output, cell_state= cell_state)
    output = tf.nn.dropout(output, keep_prob=1.0-dropout)
    outputs.append(output)
    print("the predicted state:", output)
    print("the real state:", labels[i])
  
  # calculat the MSE
  loss = tf.reduce_mean(tf.square(tf.reshape(outputs,[-1]) - tf.reshape(labels,[-1])))
  

  return loss

